In [ ]:
'''
# 현재 페이지
page_num = 1
page_ctl = 3

write_dt_lst = []
item_nm_lst = []
content_lst = []


# 날짜 
date_cut = (datetime.now() - timedelta(days = 365)).strftime('%Y%m%d')

while True :
    print(f'start : {page_num} page 수집 중, page_ctl:{page_ctl}')
    
    # 1. 셀레니움으로 html가져오기
    html_source = driver.page_source
    
    # 2. bs4로 html 파싱
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(0.5)

    # 3. 리뷰 정보 가져오기
    reviews = soup.findAll('li', {'class': 'BnwL_cs1av'})


    # 4. 한페이지 내에서 수집 가능한 리뷰 리스트에 저장
    for review in range(len(reviews)):

        # 4-1.리뷰작성일자 수집
        write_dt_raw = reviews[review].findAll('span' ,{'class' : '_2L3vDiadT9'})[0].get_text()
        write_dt = datetime.strptime(write_dt_raw, '%y.%m.%d.').strftime('%Y%m%d')

        # 4-2.상품명 수집
        # 4-2-(1) 상품명이 포함된 css 선택자 입력 
        item_nm_info_raw = reviews[review].findAll('div', {'class' : '_2FXNMst_ak'})[0].get_text()

        # 4-2-(2) re.sub() 를 활용해 dl class="XbGQRlzveO"부분부터 추출한 문장을 공백으로 대체
        item_nm_info_for_del = reviews[review].findAll('div', {'class' : '_2FXNMst_ak'})[0].find('dl', {'class' : 'XbGQRlzveO'}).get_text()

        # 4-2-(3) re.sub(pattern, replacement, string) : string에서 pattern에 해당하는 부분을 replacement로 모두 대체
        item_nm_info= re.sub(item_nm_info_for_del, '', item_nm_info_raw)

        # 4-2-(4) find() : 문자열 순서 (인덱스) 반환 : find()를 활용해 '제품 선택 : '이 나오는 인덱스 반환
        str_start_idx = re.sub(item_nm_info_for_del, '', item_nm_info_raw).find('제품 선택: ')

        # 4-2-(5) 제품명만 추출. strip(): 공백 제거 
        item_nm = item_nm_info[str_start_idx + 6:].strip()


        # 4-3. 리뷰내용 수집
        review_content_raw = reviews[review].findAll('div', {'class' : '_1kMfD5ErZ6'})[0].find('span', {'class' : '_2L3vDiadT9'}).get_text()
        review_content = re.sub(' +', ' ',re.sub('\n',' ',review_content_raw ))

        # 4-4. 수집데이터 저장
        write_dt_lst.append(write_dt)
        item_nm_lst.append(item_nm)
        content_lst.append(review_content)
 
    # 리뷰 수집일자 기준 데이터 확인(최근 1년치만 수집)
    if write_dt_lst[-1] < date_cut :
        break
        
    # page 이동
    driver.find_element(By.CSS_SELECTOR,f'#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > div > div > a:nth-child({page_ctl})').click()    
    time.sleep(5)
    
    # 셀레니움으로 html가져오기
    html_source = driver.page_source
    
    # bs4로 html 파싱
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(0.5)
 
    page_num += 1
    page_ctl += 1
 
    if page_num % 10 == 1 :
        page_ctl = 3
print('done') 
'''

In [2]:
!pip install selenium webdriver_manager

In [1]:
import requests
import time
import re
import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime, timedelta

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
options = webdriver.ChromeOptions()  # 크롬 옵션 객체 생성
options.add_argument("window-size-1920x1080")  # 전체화면
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--no-sandbox")

In [3]:
chrome_driver_path = "C:/Users/ksp45/Desktop/chromedriver-win64/chromedriver-win64/chromedriver.exe"
service = Service(executable_path = chrome_driver_path)

In [662]:
driver = webdriver.Chrome(service = service, options = options)
driver.implicitly_wait(3)

In [663]:
driver.get("https://smartstore.naver.com/ivysmall/products/8495493369")
time.sleep(3)

In [664]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)

In [665]:
driver.find_element(By.CSS_SELECTOR, "#content > div > div._3CTsMZymJs > div:nth-child(3) > div._27jmWaPaKy > ul > li:nth-child(2) > a").click()
time.sleep(5)

In [666]:
driver.find_element(By.CSS_SELECTOR, "#REVIEW > div > div._2LvIMaBiIO > div._3aC7jlfVdk > div._1txuie7UTH > ul > li:nth-child(2) > a").click()
time.sleep(3)

In [667]:
# 현재 페이지
page_num = 1
page_ctl = 3

write_dt_lst = []
item_nm_lst = []
content_lst = []

# 날짜 
date_cut = (datetime.now() - timedelta(days=365)).strftime('%Y%m%d')

while True:
    if page_num == 26:
        print("500개 수집 완료")
        break
        
    print(f'start : {page_num} page 수집 중, page_ctl:{page_ctl}')
    
    # 1. 셀레니움으로 html 가져오기
    html_source = driver.page_source

    # 2. bs4로 html 파싱
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(0.5)

    # 3. 리뷰 정보 가져오기
    reviews = soup.findAll('li', {'class': 'BnwL_cs1av'})

    # 4. 한 페이지 내에서 수집 가능한 리뷰 리스트에 저장
    for review in range(len(reviews)):
        try:
            # 4-1. 리뷰 작성일자 수집
            write_dt_raw = reviews[review].findAll('span', {'class': '_2L3vDiadT9'})[0].get_text()
            write_dt = datetime.strptime(write_dt_raw, '%y.%m.%d.').strftime('%Y%m%d')
        except Exception:
            write_dt = ''
        
        # 4-2. 상품명 수집
        try:
            item_nm_divs = reviews[review].findAll('div', {'class': '_2FXNMst_ak'})
            if item_nm_divs:
                item_nm_info_raw = item_nm_divs[0].get_text()

                # dl 태그 안의 텍스트 추출 (없을 수도 있음)
                dl_tag = item_nm_divs[0].find('dl', {'class': 'XbGQRlzveO'})
                item_nm_info_for_del = dl_tag.get_text() if dl_tag else ''

                # 텍스트 정제
                item_nm_info = re.sub(item_nm_info_for_del, '', item_nm_info_raw)

                # '제품 선택: ' 위치 찾기
                str_start_idx = item_nm_info.find('제품 선택: ')
                if str_start_idx != -1:
                    item_nm = item_nm_info[str_start_idx + len('제품 선택: '):].strip()
                else:
                    item_nm = item_nm_info.strip()
            else:
                item_nm = ''
        except Exception:
            item_nm = ''
        
        # 4-3. 리뷰내용 수집
        try:
            review_div = reviews[review].findAll('div', {'class': '_1kMfD5ErZ6'})
            if review_div:
                span_tag = review_div[0].find('span', {'class': '_2L3vDiadT9'})
                if span_tag:
                    review_content_raw = span_tag.get_text()
                    review_content = re.sub(' +', ' ', re.sub('\n', ' ', review_content_raw))
                else:
                    review_content = ''
            else:
                review_content = ''
        except Exception:
            review_content = ''
        
        # 4-4. 수집데이터 저장
        write_dt_lst.append(write_dt)
        item_nm_lst.append(item_nm)
        content_lst.append(review_content)

    # 5. 리뷰 수집일자 기준 데이터 확인 (최근 1년치만 수집)
    if write_dt_lst and write_dt_lst[-1] < date_cut:
        break

    # 6. 페이지 이동
    try:
        driver.find_element(By.CSS_SELECTOR, f'#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > div > div > a:nth-child({page_ctl})').click()
        time.sleep(5)
    except Exception as e:
        print(f"페이지 이동 실패: {e}")
        break

    page_num += 1
    page_ctl += 1
    if page_num % 10 == 1:
        page_ctl = 3

print('done')

start : 1 page 수집 중, page_ctl:3
start : 2 page 수집 중, page_ctl:4
start : 3 page 수집 중, page_ctl:5
start : 4 page 수집 중, page_ctl:6
done


In [668]:
result_df = pd.DataFrame({
              'Item_info' : item_nm_lst,
              'Content' : content_lst,
              'Date' : write_dt_lst })

In [669]:
print(result_df["Date"])
print(len(result_df))

0     20250721
1     20250719
2     20250719
3     20250719
4     20250713
        ...   
75    20240524
76    20240516
77    20240515
78    20240514
79    20240510
Name: Date, Length: 80, dtype: object
80


In [670]:
result_df.to_csv('C:/Users/ksp45/Desktop/Zinc_Ivy.csv', index = None, encoding = 'utf-8-sig')